# Пример решения соревнования по линейным моделям

## Импорт необходимых библиотек

Импортируем библиотеку pandas для работы с табличными данными

In [2]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

# Подгрузка данных

In [3]:
!pip install gdown

     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     ----------------------------------- -- 133.1/143.0 kB 4.0 MB/s eta 0:00:01
     -------------------------------------- 143.0/143.0 kB 2.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\sudo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
!gdown https://drive.google.com/uc?id=1p1HtCvxqnDNfk52_Pcli6IlTFxtlXLUK
!gdown https://drive.google.com/uc?id=1ZinEL6IyBvxzygqUfjqZQsTeZMC7mBm4
!gdown https://drive.google.com/uc?id=1xcm2nFUykNqau4IqYLy3Ri_9OcurZnZn

Downloading...
From: https://drive.google.com/uc?id=1p1HtCvxqnDNfk52_Pcli6IlTFxtlXLUK
To: d:\Papa\digital_teams\regression\train_set.csv

  0%|          | 0.00/807k [00:00<?, ?B/s]
 65%|██████▍   | 524k/807k [00:00<00:00, 4.85MB/s]
100%|██████████| 807k/807k [00:00<00:00, 6.12MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZinEL6IyBvxzygqUfjqZQsTeZMC7mBm4
To: d:\Papa\digital_teams\regression\test_set.csv

  0%|          | 0.00/180k [00:00<?, ?B/s]
100%|██████████| 180k/180k [00:00<00:00, 3.05MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xcm2nFUykNqau4IqYLy3Ri_9OcurZnZn
To: d:\Papa\digital_teams\regression\ford_price_kaggle_sample_submission.csv

  0%|          | 0.00/87.2k [00:00<?, ?B/s]
100%|██████████| 87.2k/87.2k [00:00<00:00, 2.66MB/s]


In [5]:
df = pd.read_csv('train_set.csv')
df.head()

,automobile_id,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,0,Fiesta,2018,12000,Manual,30418,Petrol,145,62.8,1.0
1,1,Fiesta,2016,9790,Manual,35047,Diesel,0,78.5,1.5
2,2,Edge,2018,32000,Automatic,11007,Diesel,145,37.2,2.0
3,3,Kuga,2017,17595,Semi-Auto,18066,Diesel,145,54.3,2.0
4,4,Focus,2017,9998,Manual,34025,Petrol,20,60.1,1.0


Наши данные содержат следующие признаки

- **automobile_id** - идентификатор автомобиля;
- **model** - Бренд автомобиля Ford Ford Car Brands;
- **year** - Год выпуса;
- **price** - Цена автомобиля. *Целевая переменная*;
- **transmission** - Тип коробки передач: Automatic (автоматическая), Manual (ручная), Semi-Auto (полуавтомат);
- **mileage** -> Пробег в милях;
- **fuel_Type** -> Вид топлива: Petrol (бензин), Diesel (дизель), Hybrid (гибрид), Electric (электроавтомобиль), Other (другое);
- **tax** -> Размер налога на автомобиль;
- **mpg** - Потребление топлива: галлоны на милю;
- **engineSize** - Объем двигателя.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14368 entries, 0 to 14367
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   automobile_id  14368 non-null  int64  
 1   model          14368 non-null  object 
 2   year           14368 non-null  int64  
 3   price          14368 non-null  int64  
 4   transmission   14368 non-null  object 
 5   mileage        14368 non-null  int64  
 6   fuelType       14368 non-null  object 
 7   tax            14368 non-null  int64  
 8   mpg            14368 non-null  float64
 9   engineSize     14368 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 1.1+ MB


In [7]:
df.nunique()

automobile_id    14368
model               19
year                21
price             3122
transmission         3
mileage          11236
fuelType             5
tax                 34
mpg                 85
engineSize          15
dtype: int64

## Подготовка данных

Переведем наши категории в числовые значения, так как линейные модели не умеют работать с категориями напрямую. Сохраним енкодеры в словарь чтобы потом переиспользовать их.

In [8]:
cat_cols = ['model', 'transmission', 'fuelType']

dict_encoders = dict()

for col in cat_cols:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
    dict_encoders[col] = encoder

df.head()

,automobile_id,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,0,4,2018,12000,1,30418,4,145,62.8,1.0
1,1,4,2016,9790,1,35047,0,0,78.5,1.5
2,2,3,2018,32000,0,11007,0,145,37.2,2.0
3,3,12,2017,17595,2,18066,0,145,54.3,2.0
4,4,5,2017,9998,1,34025,4,20,60.1,1.0


Разделим данные на обучающую и тестовую выборку

In [9]:
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Обучение модели и предсказание

В качестве бейзлайна используем модель линейной регрессии

In [10]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [11]:
print('Train absolute error: ', mae(y_train, y_pred_train))
print('Test absolute error: ', mae(y_test, y_pred_test))

Train absolute error:  1759.5306170597137
Test absolute error:  1741.1707074961048


## Подготовка финального предсказания

Подгрузим тестовую выборку и шаблон для отправки данных

In [12]:
df_test = pd.read_csv('test_set.csv')
submit_data = pd.read_csv('ford_price_kaggle_sample_submission.csv')
df_test.head()

,automobile_id,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,0,Focus,2013,Manual,47781,Petrol,30,56.5,1.0
1,1,Galaxy,2019,Semi-Auto,3763,Diesel,145,54.3,2.0
2,2,Fiesta,2017,Manual,47959,Diesel,0,78.5,1.5
3,3,Fiesta,2010,Manual,99000,Petrol,145,49.6,1.2
4,4,Fiesta,2018,Manual,13820,Petrol,145,65.7,1.0


Чтобы для предсказания на новом датасете необходимо повторить все шаги, совершенные нами ранее. Важно использовать те же методы подготовки данных и обученные модели.

Используем уже имеющиеся енкодеры для преобразования категориальных переменных

In [13]:
for col in cat_cols:
    encoder = dict_encoders[col]
    df_test[col] = encoder.transform(df_test[col])

df_test.head()

,automobile_id,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,0,5,2013,1,47781,4,30,56.5,1.0
1,1,7,2019,2,3763,0,145,54.3,2.0
2,2,4,2017,1,47959,0,0,78.5,1.5
3,3,4,2010,1,99000,4,145,49.6,1.2
4,4,4,2018,1,13820,4,145,65.7,1.0


Сделаем предсказание уже обученной моделью и сохраним его в файл для отправки

In [14]:
y_pred = model.predict(df_test)
submit_data['price'] = y_pred

submit_data.to_csv('my_submisson.csv', index=False)

## Идеи для улучшения
 - изменить способ подготовки данных
 - использовать модель с другими параметрами
 - подобрать гиперпараметры модели